# ICE 1 - Using MongoDB in Julia

**Author**: Nicolas Dussaillant

## Preparation

For this code, the computer specifications were the following:
- Processor: Apple Silicon M1
- OS: MacOS 11.6 (Big Sur)

Previously installed software used for installations and usage:
- [Homebrew](https://docs.brew.sh/)
- [Anaconda](https://docs.anaconda.com/anaconda/install/mac-os/)

### Installing and preparing Julia:

1. **Download and install Julia:**
    
    For native operation of Julia it is necessary to install v1.7 which is not available through Homebrew, so download and install version 1.7.1 for Apple M-Series [here](https://julialang.org/downloads/) (experimental version).


2. **Create a link to execute Julia from the Terminal:**
    
    Run the following link in the terminal:

    `sudo ln -s /Applications/Julia-1.7.app/Contents/Resources/julia/bin/julia /usr/local/bin/julia`

    (Make sure that directory `/usr/local/bin` exists or create it before running that command)


3. **Download IJulia to use Jupyter Notebooks:**

    Enter Julia typing `julia` in the Terminal. Then press `]`, type `add IJulia` and hit **enter**. After installation, hit **delete** and exit Julia (running `exit()` or pressing `ctrl + D`).
    
    After that, run Jupyter Notebook from Anaconda Navigator, and create a new Julia notebook.
    
    
### Installing and preparing MongoDB

For MongoDB we will use HomeBrew. All the following is run in the **Terminal**.

1. **Add third-party repositories for MongoDB in Brew.** Run:

    `brew tap mongodb/brew`
    
    
2. **Install mongodb-community running**:

    `brew install mongodb-community`
    
    
3. **To start and stop mongodb, use the following commands (although we will run them in this Notebook as well)**:

    `brew services start mongodb-community`

    `brew services stop mongodb-community`
    
    
**Note**: To edit configurations of MongoDB, edit the following file: `/opt/homebrew/etc/mongod.conf`. By default, MongoDB will run on 127.0.0.1:27017
